In [1]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from environments import RodentSingleClipTrack
import numpy as np
import mediapy as media
import jax.numpy as jp

params = {
    "scale_factor": .9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 3,
    "clip_path": "12_22_1_250_clip_0.p",
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
    ],
}

# env objects

In [2]:
env = RodentSingleClipTrack(params)

In [3]:
key = random.PRNGKey(0)
jit_step = jax.jit(env.step)
jit_reset = jax.jit(env.reset)

In [4]:
n = env.reset(key)


In [ ]:
n = env.step(n, random.uniform(key, shape=(env.sys.nu,)))

/home/charles/github/VNL-Brax-Imitation/mocap_preprocess.py:325: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree.leaves (jax v0.4.25 or newer) or jax.tree_util.tree_leaves (any JAX version).
  leaves = jax.tree_leaves(self)


In [7]:
n.done

<bound method RodentSingleClipTrack._calculate_termination of <environments.RodentSingleClipTrack object at 0x7f4de3237010>>

In [25]:
next = jit_reset(key)
print("starting rollout")
mu = 0
sigma = .1
rollout = []
for _ in range(200):
    next = jit_step(next, mu + sigma * random.normal(key, shape=(env.sys.nu,)))
    rollout.append(next)


starting rollout


In [29]:
import mediapy as media
import os

os.environ["MUJOCO_GL"] = "glfw"
rollout_data = [s.pipeline_state for s in rollout]

video = env.render(rollout_data, camera='close_profile', height=500, width=500)

media.show_video(video, fps=1.0 / env.dt)

This browser does not support the video tag.


In [39]:
jax.tree_util.tree_map(
    lambda x: print(x.shape),
    env._ref_traj,
)

(3, 250)
(15, 250)
(54, 250)
(72, 250)
(3, 250)
(12, 250)
(67, 250)
(67, 250)
(0,)
(3, 250)
(4, 250)
(0,)
(3, 250)


ReferenceClip(angular_velocity=None, appendages=None, body_positions=None, body_quaternions=None, center_of_mass=None, end_effectors=None, joints=None, joints_velocity=None, markers=None, position=None, quaternion=None, scaling=None, velocity=None)

In [5]:
env._ref_traj.body_positions[:,env.body_idxs]

array([[-5.15703410e-02, -5.44421673e-02, -5.97836301e-02,
        -6.21297993e-02, -5.42572998e-02, -5.62890023e-02,
        -5.67587502e-02, -5.36332317e-02, -5.16615808e-02,
        -4.88138646e-02, -4.33837250e-02, -3.61943245e-02,
        -3.10629196e-02, -2.80833934e-02, -2.56768093e-02,
        -2.37950906e-02, -1.97961070e-02, -1.57608576e-02],
       [ 3.40509087e-01,  3.44985902e-01,  3.55930865e-01,
         3.62414867e-01,  3.66985410e-01,  3.72000068e-01,
         3.81666929e-01,  3.86683911e-01,  3.88806075e-01,
         3.91307801e-01,  3.96168560e-01,  4.00209337e-01,
         4.04788226e-01,  4.09968972e-01,  4.16549325e-01,
         4.23274040e-01,  4.28765714e-01,  4.32083309e-01],
       [ 7.10330382e-02,  7.15883821e-02,  7.32645169e-02,
         7.36922920e-02,  7.38230348e-02,  7.33438656e-02,
         7.03054219e-02,  7.03579783e-02,  7.04110041e-02,
         6.96249455e-02,  7.07218349e-02,  7.11583570e-02,
         7.14595839e-02,  7.19613433e-02,  7.33096227e

In [6]:
env._ref_traj.angular_velocity.shape

(3, 250)

In [7]:
# def slice_arrays(start_index, end_index):
#     def slicer(x):
#         if isinstance(x, jp.ndarray):
#             return x[..., start_index:end_index]
#         return x
#     return slicer

# sliced_tree = jax.tree_map(slice_arrays(0, 5), env._ref_traj)
# sliced_tree.body_positions.shape

In [8]:
start_index = 0
end_index = 5
sliced_clip = env._ref_traj.slice_clip(start_index, end_index)
sliced_clip.center_of_mass.shape

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/environments.py:41: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(slicer, self)


(3, 5)

# state objects & reset_fn, get_obs_fn

In [10]:
rng = random.PRNGKey(0)
state = env.reset(rng)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx

In [11]:
state.info['start_frame']

Array(56, dtype=int32)

In [12]:
env._ref_traj.velocity[:, state.info['start_frame']]

array([ 0.40821564, -0.20567179, -0.11344627])

In [13]:
state.pipeline_state.xpos.shape

(66, 3)

In [14]:
env._ref_traj.body_positions.shape

(54, 250)

In [15]:
state.obs.shape

(1735,)

end effector position

In [25]:
state.pipeline_state.xpos[jp.array(env._end_eff_idx)]

Array([[-0.00912952,  0.4411053 ,  0.01141115],
       [-0.03021844,  0.414083  ,  0.01621009],
       [ 0.0780039 ,  0.40200663,  0.01433735],
       [ 0.08267315,  0.36600125,  0.02249428]], dtype=float32)

In [28]:
env._ref_traj.end_effectors.shape

(12, 250)

# step_fn, reward_fn, & termination_fn

In [16]:
# rollout = [state.pipeline_state]
# ctrl = jax.numpy.array(np.random.uniform(-1,1, env.sys.nu))
# state = env.step(state, ctrl)
# rollout.append(state.pipeline_state)

In [17]:
# media.show_video(env.render(rollout,camera='close_profile'), fps=1.0 / env.dt)